In [1]:
from sage.misc.prandom import randrange

Par=1
Nsig=1500
print("Par, # signatures", Par, Nsig)

if Par==1:
    n=4096
    k=539
    l=4
    np=1024 #n'
    kp=890 #k'
    w1=31
    w2=531    
if Par==2:
    n=8192
    k=1065
    l=8
    np=1024
    kp=880
    w1=53
    w2=807
kpl=kp*l
lnk=l*(np-kp)

#generate nrow x ncol matrices over GF(2)
def GenRandom(nrow,ncol): 
    M=matrix(GF(2),nrow,ncol)
    for i in range(nrow):
        for j in range(ncol):
            M[i,j]=randrange(2)
    return M

#generate ncol x ncol permutation matrix
def GenPermMat(ncol): # the i-th column of the identity matrix becomes the sigma(i)-th column of the permutation matrix
    sigma=Permutations(ncol).random_element() #random permutation sigma
#    print("permuting cols", sigma)
    M=matrix(GF(2),ncol,ncol)
    for i in range(ncol):
        M[i,sigma[i]-1]=1
    return M

#generate a random vector (as 1 x ncol matrix over GF(2)) of weight wt
def GenVecWt(ncol,wt):
    coord = sample(range(ncol), wt)
    v=matrix(GF(2),1,ncol)
    for i in coord:
        v[0,i]=1
    return v

############################### Part 0: Key Generation ###############################

H=GenRandom(n-k,n)
I=identity_matrix(GF(2), kp)
Esub=[] #list containing the submatrices E_i = (I|R_i)
for i in range(l):
    R=GenRandom(kp,np-kp)
    Ei=block_matrix(1,2,[ I, R ])
    Esub.append(Ei)    
E0=block_matrix(1,l,Esub)
P1=GenPermMat(kp)
P2=GenPermMat(n)
E= P1*E0*P2
S=H*(E.transpose())
#we now have H,S,E (as well as P1,P2)
############################ End of Key Generation #########################

############################## Collecting Signatures #########################
#generate a signature
def Sign():
    e=GenVecWt(n,w2)
    c=GenVecWt(kp,w1)
    z=c*E+e
    T=[]
    for i in range(n):
        T.append(ZZ(z[0][i]))
    for i in range(kp):
        T.append(ZZ(c[0][i]))
    return T

#collecting Nsig signatures
siglist=[]
for i in range(Nsig):
    sig=Sign()
    siglist.append(sig)
    
print("Starting key recovery attack")
    
######################### Part 1: Classifying Columns of E ####################    
Ii=[] #index of the identity columns
Ir=[] #index of the random columns  
timestart=cputime(subprocesses=True)
N=250
delta=0.3
deltaN=delta*N
for j in range(n):
    s=0
    for i in range(N):
        s=s+siglist[i][j] #z_j of the i-th signature
    if s < deltaN:
        Ii.append(j)
    else:
        Ir.append(j)
timeplus=cputime(subprocesses=True) - timestart
print("time1", timeplus)

############################### Part 2: Determining the Identity Part of E ##############
Eguess=matrix(GF(2),kp,n)

timestart=cputime(subprocesses=True)
delta=0.5
LenIi=len(Ii)
for i0 in range(kp):  
    siglistRestricted=[]
    for sig in siglist:
        if sig[n+i0] == 1: #checking whether c_{i0}=1
            siglistRestricted.append(sig)
    N=len(siglistRestricted)
    deltaN=delta*N
    
    for j in range(kpl):
        s=0
        jindex=Ii[j]
        for sig in siglistRestricted:
            s=s+sig[jindex]
        if s >= deltaN:
            Eguess[i0,jindex]=1
timeplus=cputime(subprocesses=True) - timestart
print("time2", timeplus)

##################### Part 3: Solving the Remaining Entries (Random Part) of E ############# 

timestart=cputime(subprocesses=True)
Hsub=matrix(GF(2),lnk,lnk)
i=0
i0=0
UsefulRow=[]
while(i<lnk): #obtaining l(n'-k') x l(n'-k') invertible submatrix Hsub and computing its inverse HsubInv
    for j in range(lnk):
        Hsub[i,j]=H[i0,Ir[j]]
    if Hsub.rank()==i+1:
        UsefulRow.append(i0)
        i=i+1
    i0=i0+1
Hsub=Hsub.inverse() #Hsub^(-1)
timeplus=cputime(subprocesses=True) - timestart
print("time3.1", timeplus)

timestart=cputime(subprocesses=True)
y=vector(GF(2),lnk)
for i0 in range(kp): #should be in range(kp)        
    timeStart0=cputime(subprocesses=True)
    EG=[]
    for i in range(n):
        EG.append(0)
    for j in Ii:
        EG[j]=Eguess[i0,j]
    for i in range(lnk):
        iu=UsefulRow[i]
        t1=S[iu,i0] 
        for j in Ii:
            t1=t1-H[iu,j]*EG[j]  
        y[i]=t1
    x=Hsub*y
    for i in range(lnk):
        Eguess[i0,Ir[i]]=x[i]

timeplus=cputime(subprocesses=True) - timestart
print("time3", timeplus)

print("key recovery is done")

def CheckE():
    for i in range(kp):
        for j in range(n):
            if E[i,j] != Eguess[i,j]:
                print("E[i,j] false at", i,j)
                return false
    return true

print("The recovered secret key is", CheckE())

print("done")

Par, # signatures 1 1500
Starting key recovery attack
time1 0.22853100000000381
time2 29.520818999999996
time3.1 0.29642599999999675
time3 584.134697
key recovery is done
The recovered secret key is True
done
